In [ ]:
    !pip install -U langchain
!pip install -U "langchain[google-genai]"
# Requires Python 3.10+

  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.0.7
    Uninstalling langchain-core-1.0.7:
      Successfully uninstalled langchain-core-1.0.7
  Attempting uninstall: langchain━━━━━━━━━━━━━━━ 0/2 [langchain-core]
    Found existing installation: langchain 1.0.8 0/2 [langchain-core]
    Uninstalling langchain-1.0.8:━━━━━━━━━━━ 0/2 [langchain-core]
      Successfully uninstalled langchain-1.0.80m 0/2 [langchain-core]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [langchain]]


UnboundLocalError: cannot access local variable 'child' where it is not associated with a value

--- Logging error ---
Traceback (most recent call last):
  File "/home/hoangnam/miniconda3/lib/python3.13/site-packages/IPython/utils/_process_posix.py", line 125, in system
    child = pexpect.spawn(self.sh, args=['-c', cmd])  # Vanilla Pexpect
  File "/home/hoangnam/miniconda3/lib/python3.13/site-packages/pexpect/pty_spawn.py", line 205, in __init__
    self._spawn(command, args, preexec_fn, dimensions)
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hoangnam/miniconda3/lib/python3.13/site-packages/pexpect/pty_spawn.py", line 303, in _spawn
    self.ptyproc = self._spawnpty(self.args, env=self.env,
                   ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
                                 cwd=self.cwd, **kwargs)
                                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/hoangnam/miniconda3/lib/python3.13/site-packages/pexpect/pty_spawn.py", line 315, in _spawnpty
    return ptyprocess.PtyProcess.spawn(args, **kwargs)
           ~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
import ast
import json

input_file = "/home/hoangnam/fusion/data/meta_baby.json"     # file bạn chứa các dict
output_file = "products.json"       # file JSON hoàn chỉnh muốn tạo

products = []

with open(input_file, "r", encoding="utf-8") as f:
    # Nếu mỗi product là 1 dòng => duyệt từng dòng
    for line in f:
        line = line.strip()
        if not line:
            continue  # bỏ qua dòng trống
        # Chuyển từ chuỗi dict kiểu Python sang object Python
        obj = ast.literal_eval(line)
        products.append(obj)

# Ghi ra file JSON chuẩn: là 1 mảng các object
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(products, f, ensure_ascii=False, indent=2)

print("Đã tạo file", output_file)


Đã tạo file products.json


In [ ]:
import json

# -------------------------------
# 1. Load model file (VLM outputs)
# -------------------------------
with open("amazon_baby_model_qwen_type_sample_descriptions.json", "r") as f:
    model_data = json.load(f)

# -------------------------------
# 2. Load products.json (CHUẨN JSON)
# -------------------------------
with open("/home/hoangnam/fusion/products.json", "r") as f:
    products = json.load(f)

# -------------------------------
# 3. Map asin → original_description
# -------------------------------
product_map = {
    item["asin"]: item.get("description", "")
    for item in products
}

# -------------------------------
# 4. Merge vào model_data
# -------------------------------
merged = []
missing = []

for item in model_data:
    asin = item.get("asin")
    if asin in product_map:
        item["original_description"] = product_map[asin]
        merged.append(item)
    else:
        missing.append(asin)

# -------------------------------
# 5. Save file merge (optional)
# -------------------------------
with open("merged_products.json", "w") as f:
    json.dump(merged, f, indent=2)

print("Merged items:", len(merged))
print("Missing ASINs:", missing)


Merged items: 7036
Missing ASINs: []


In [ ]:

import json
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek  # 👈 DeepSeek integration


# ---------------------------
# SETUP LLM + PARSER (DEEPSEEK)
# ---------------------------
DEEPSEEK_API_KEY = os.environ.get("DEEPSEEK_API_KEY")  # or hardcode for quick test

llm = ChatDeepSeek(
    model="deepseek-chat",                 # or "deepseek-reasoner" if you want CoT
    api_key=DEEPSEEK_API_KEY,
    api_base="https://api.deepseek.com",
    temperature=0,
)

parser = StrOutputParser()




# ---------------------------
# PHASE 1: ANALYSIS
# ---------------------------
analysis_prompt = PromptTemplate(
    input_variables=["title", "img_title_desc", "original_desc"],
    template="""
You are an information extraction assistant.

Product title:
{title}

Descriptions:
1. Image-title-based description: {img_title_desc}
2. Original Amazon description: {original_desc}

Your tasks:
1. Identify all common correct details appearing in both descriptions or implied by the title.
2. Identify useful unique details from each description.
3. Identify contradictions between them.

Return JSON EXACTLY:

{{
  "common_details": [...],
  "unique_details": {{
    "img_title_desc": [...],
    "original_desc": [...]
  }},
  "contradictions": [...]
}}
"""
)
analysis_chain = analysis_prompt | llm | parser


rewrite_prompt = PromptTemplate(
    input_variables=["title", "analysis", "feedback"],
    template="""
You are a product description rewriting assistant.

Product title:
{title}

Here is a JSON analysis of this product:
{analysis}

Here is feedback from fact-checking (if any):
{feedback}

RULES ABOUT SOURCE PRIORITY:
- The product title defines the core product identity.
- The original description is the most authoritative source of factual information.
- The image_title_desc may be used for tone, phrasing, and soft implications, but NOT to override or contradict the original_desc.
- The image_based_desc should generally NOT be used for factual claims unless it clearly matches the product type.

Your job:
Write or revise ONE unified product description (1–3 sentences) that:
- is clearly about the product in the title,
- preserves facts from the original_desc first,
- may use wording or sentiment from img_title_desc to make the description more natural,
- removes or softens unsupported or conflicting claims,
- does NOT invent new facts or assume details.

CRITICAL RULES FOR CONFLICTING INFORMATION:
- If the analysis lists a contradiction, do not choose one side. Use neutral grouping phrasing.
  Example:
    WRONG: “includes a pacifier, gum massager, and a teether”
    CORRECT: “includes multiple oral care tools designed for early dental hygiene”

- If the analysis lists a contradiction in material:
    WRONG: “100% cotton”
    WRONG: “cotton blend”
    CORRECT: “cotton fabric”
    CORRECT: “soft cotton material”

Fallback rules:
- If original_desc is missing or extremely short, you may rely more on img_title_desc but keep claims conservative and neutral.
- If there is uncertainty, write a safer, more general phrasing rather than a strong factual claim.

Requirements:
- Start the description with the exact product name from the title.
- Make the text natural and suitable for an online product listing.
- Apply feedback faithfully: if a detail was flagged as unsupported, do NOT reintroduce it.

Return ONLY the final product description as plain text.
"""
)

rewrite_chain = rewrite_prompt | llm | parser



reflection_prompt = PromptTemplate(
    input_variables=["title", "draft", "img_title_desc", "original_desc"],
    template="""
You are a fact-checking and consistency validation agent.

Product title:
{title}

Draft fused description:
{draft}

Allowed sources for verification (in descending reliability):
Tier 1 (highest trust): Product title
Tier 2: Original description
Tier 3: Image_title_desc (linguistic tone & soft implications, NOT technical facts)
Tier 4 (lowest trust): image_based_desc — use only for confirming object category, NOT specific details.

Your goal is NOT to be overly strict, but to verify that the draft is:
- factually safe,
- respects the source priority order,
- conflict-neutral when sources disagree,
- and does not assert unsupported strong claims.

RULES:

1) A statement in the draft is SUPPORTED if it is:
- explicitly stated in the original_desc or title, OR
- clearly implied by them, OR
- consistent with img_title_desc phrasing (but NOT contradictory to original_desc), OR
- a neutral/general phrasing that does not choose sides in a contradiction.

2) If the sources contradict each other about some detail (such as included components or material), then:
- Strong specific claims are UNSUPPORTED.
- Neutral grouping phrasing like “includes several items for…” is ACCEPTABLE.

3) The image_based_desc is NOT sufficient to reject or validate facts UNLESS it contradicts product identity in title.
Example:
title: toothbrush
image shows: smartphone
→ image should be ignored

4) Only flag a FAIL if:
- the draft contains a strong factual claim
- AND that exact claim is NOT supported by the original_desc or title,
- AND the claim is not a neutral/general phrasing,
- AND the claim is not simply stylistic paraphrasing.

5) You MUST NOT fail the draft:
- for neutral phrasing,
- or for safe paraphrasing,
- or for stylistic differences,
- or for soft/implicit wording of common-sense facts,
- or for minor differences in tone or emphasis.

OUTPUT FORMAT:

If all facts in the draft are supported or neutrally phrased → return EXACTLY:

OK

Otherwise return:

FAIL: [list of specific exact problematic phrases]

where each phrase appears literally in the draft text.
"""
)

reflection_chain = reflection_prompt | llm | parser








# ---------------------------
# TEST ITEM
# ---------------------------
item = {
  "asin": "XTEST02",
  "title": "Summer Infant 3 Piece Oral Care Set",
  "image_title_based_desc": "Includes soft-bristled toothbrush, teething ring, and pacifier, designed for oral development from infancy.",
  "original_description": "Kit includes a 3 stage oral care system: gum massager, finger toothbrush, and toddler toothbrush. Helps parents maintain oral hygiene.",
  "image_based_desc": "The product is a small boxed kit containing multiple small plastic baby care items.",
  "llm_based_desc": "The Summer Infant 3 Piece Oral Care Set features teethers and pacifiers for soothing, along with a toothbrush designed for baby gums."
}



import re
import json

def extract_json(text: str) -> str:
    """
    Remove ```json ... ``` or ``` ... ``` fences and return the inner JSON string.
    """
    text = text.strip()
    # If it starts with ``` something, strip the code fence
    if text.startswith("```"):
        # Remove leading ```... and trailing ```
        text = re.sub(r"^```[a-zA-Z]*\n?", "", text)   # remove ```json\n or ```\n
        text = re.sub(r"\n?```$", "", text.strip())   # remove closing ```
    return text.strip()

# ---------------------------
# SIMPLE TEST PIPELINE
# ---------------------------
def run_deepseek_test(item, max_loops=2):
    title = item["title"]
    img_title_desc = item["image_title_based_desc"]
    original_desc = item["original_description"]

    print("\n==============================")
    print(f"RUNNING ITEM: {title}")
    print("==============================")

    # ---- Phase 1: ANALYSIS ----
    analysis_raw = analysis_chain.invoke({
    "title": title,
    "img_title_desc": img_title_desc,
    "original_desc": original_desc
    })
    print("\n--- PHASE 1 ANALYSIS (RAW) ---")
    print(analysis_raw)

    try:
        cleaned = extract_json(analysis_raw)
        analysis_json = json.loads(cleaned)
    except json.JSONDecodeError:
        print("\n[WARN] DeepSeek returned invalid JSON even after cleaning. Using raw string for next phase.")
        analysis_json = analysis_raw


    feedback = "No feedback yet. This is the first draft."
    last_draft = None

    # ---- Looped REWRITE ↔ REFLECTION ----
    for i in range(max_loops):
        print(f"\n=== LOOP TURN {i+1} ===")

        # Phase 2: REWRITE
        draft = rewrite_chain.invoke({
            "title": title,
            "analysis": analysis_json if isinstance(analysis_json, str) else json.dumps(analysis_json),
            "feedback": feedback,
        })
        print("\n--- PHASE 2 DRAFT ---")
        print(draft)
        last_draft = draft

        # Phase 3: REFLECTION
        reflection = reflection_chain.invoke({
            "title": title,
            "draft": draft,
            "img_title_desc": img_title_desc,
            "original_desc": original_desc
        })
        print("\n--- PHASE 3 REFLECTION ---")
        print(reflection)

        if reflection.strip().startswith("OK"):
            print("\n✅ Reflection says OK. Stopping loop.")
            break
        else:
            feedback = reflection  # feed FAIL back into next rewrite

    return last_draft, reflection


if __name__ == "__main__":
    final_draft, final_reflection = run_deepseek_test(item)
    print("\n==============================")
    print("FINAL DRAFT:")
    print(final_draft)
    print("\nFINAL REFLECTION:")
    print(final_reflection)



In [6]:
import json
import os
import time   # ⬅️ NEW
import random # ⬅️ OPTIONAL (for random backoff)

def safe_invoke(chain, payload, max_retries=5):
    """Wrapper to call LangChain chain with retry on 429 (quota) errors."""
    attempt = 0
    while True:
        try:
            return chain.invoke(payload)
        except Exception as e:
            msg = str(e)
            # Gemini free-tier / quota / rate limit errors
            if "429" in msg or "quota" in msg.lower() or "ResourceExhausted" in msg:
                attempt += 1
                if attempt > max_retries:
                    print("⚠️ Max retries exceeded, re-raising error.")
                    raise
                # Wait 50–70s like Google suggests
                wait = random.randint(50, 70)
                print(f"⚠️ Quota exceeded (attempt {attempt}) — sleeping {wait}s...\nError: {e}")
                time.sleep(wait)
            else:
                # Not a quota error → don't hide it
                raise


In [7]:
def run_loop(item, max_loops=3):
    title = item["title"]
    asin = item.get("asin", None)  # 🔹 lấy asin từ item
    img_title_desc = item.get("image_title_based_desc", "")
    original_desc = item["original_description"]

    print("\n==============================")
    print(f"RUNNING ITEM: {title}")
    print("==============================")

    # Phase 1: phân tích 1 lần (có retry)
    analysis = safe_invoke(
        analysis_chain,
        {
            "title": title,
            "img_title_desc": img_title_desc,
            "original_desc": original_desc
        }
    )
    print("\n--- PHASE 1 ANALYSIS ---")
    print(analysis)

    feedback = "No feedback yet. This is the first draft."
    last_draft = None

    # Loop Phase 2 ↔ Phase 3
    for i in range(max_loops):
        print(f"\n=== LOOP TURN {i+1} ===")

        # Phase 2: REWRITE với feedback hiện tại (có retry)
        draft = safe_invoke(
            rewrite_chain,
            {
                "title": title,
                "analysis": analysis,
                "feedback": feedback,
            }
        )
        last_draft = draft

        print("\n--- PHASE 2 DRAFT ---")
        print(draft)

        # Phase 3: CHECK (có retry)
        reflection = safe_invoke(
            reflection_chain,
            {
                "title": title,
                "draft": draft,
                "img_title_desc": img_title_desc,
                "original_desc": original_desc
            }
        )

        print("\n--- PHASE 3 CHECK ---")
        print(reflection)

        if reflection.strip().upper().startswith("OK"):
            print("\n✅ FINAL OUTPUT:")
            print(draft)
            return {
                "asin": asin,
                "final_description": draft
            }

        feedback = reflection

    print("\n❌ Failed after max loops — returning last draft")
    return {
        "asin": asin,
        "final_description": last_draft
    }


In [8]:
def safe_invoke(chain, payload, max_retries=5):
    """Wrapper to call LangChain chain with retry on 429 (quota) errors."""
    attempt = 0
    while True:
        try:
            return chain.invoke(payload)
        except Exception as e:
            msg = str(e)
            # Gemini free-tier / quota / rate limit errors
            if "429" in msg or "quota" in msg.lower() or "ResourceExhausted" in msg:
                attempt += 1
                if attempt > max_retries:
                    print("⚠️ Max retries exceeded, re-raising error.")
                    raise
                # Wait 50–70s like Google suggests
                wait = random.randint(50, 70)
                print(f"⚠️ Quota exceeded (attempt {attempt}) — sleeping {wait}s...\nError: {e}")
                time.sleep(wait)
            else:
                # Not a quota error → don't hide it
                raise


In [12]:
import json
import os

# ===========================
# CẤU HÌNH FILE
# ===========================
INPUT_PATH = "merged_products.json"            # file input gốc
CHECKPOINT_PATH = "final_outputs_checkpoint.json"  # file save point
OUTPUT_PATH = "final_outputs.json"            # file output cuối cùng

# ===========================
# LOAD TOÀN BỘ DATASET
# ===========================
with open(INPUT_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"📂 Loaded {len(data)} items from {INPUT_PATH}")

# ===========================
# KHỞI TẠO TỪ CHECKPOINT (NẾU CÓ)
# ===========================
final_outputs = []
processed_asins = set()

if os.path.exists(CHECKPOINT_PATH):
    print(f"🔄 Found checkpoint file: {CHECKPOINT_PATH}, resuming...")
    with open(CHECKPOINT_PATH, "r", encoding="utf-8") as f:
        final_outputs = json.load(f)
    # Lấy danh sách asin đã xử lý
    for item in final_outputs:
        asin = item.get("asin")
        if asin is not None:
            processed_asins.add(asin)
    print(f"✅ Loaded {len(final_outputs)} items from checkpoint.")
else:
    print("🆕 No checkpoint found. Starting from scratch.")

# ===========================
# CHẠY PIPELINE CHO TOÀN BỘ DATA
# ===========================
total = len(data)

for idx, item in enumerate(data):
    asin = item.get("asin")

    # Bỏ qua item đã xử lý rồi (khi resume)
    if asin in processed_asins:
        print(f"[SKIP] {idx+1}/{total} - ASIN {asin} already processed.")
        continue

    print(f"\n🚀 Processing {idx+1}/{total} - ASIN: {asin}")

    try:
        out = run_loop(item)  # out = {"asin": ..., "final_description": ...}
    except Exception as e:
        print(f"❌ Error when processing ASIN {asin}: {e}")
        # Có thể log ra file lỗi nếu muốn, tạm thời bỏ qua để không dừng toàn bộ
        continue

    # Lưu vào list kết quả
    final_outputs.append(out)
    processed_asins.add(asin)

    # ===========================
    # CHECKPOINT SAU MỖI ITEM
    # ===========================
    try:
        with open(CHECKPOINT_PATH, "w", encoding="utf-8") as f:
            json.dump(final_outputs, f, indent=2, ensure_ascii=False)
        print(f"💾 Checkpoint saved ({len(final_outputs)} items) → {CHECKPOINT_PATH}")
    except Exception as e:
        print(f"⚠️ Warning: failed to write checkpoint: {e}")



# ===========================
# SAVE FILE CUỐI CÙNG
# ===========================
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    json.dump(final_outputs, f, indent=2, ensure_ascii=False)

print("\n==========================")
print("✅ FINISHED ALL ITEMS")
print("==========================")
print(f"📁 Final output saved to: {OUTPUT_PATH}")
print(f"📝 Checkpoint file (last state) kept at: {CHECKPOINT_PATH}")


📂 Loaded 7036 items from merged_products.json
🔄 Found checkpoint file: final_outputs_checkpoint.json, resuming...
✅ Loaded 7036 items from checkpoint.
[SKIP] 1/7036 - ASIN B00003TKWE already processed.
[SKIP] 2/7036 - ASIN B0000482F9 already processed.
[SKIP] 3/7036 - ASIN B00004RA66 already processed.
[SKIP] 4/7036 - ASIN B000056HM5 already processed.
[SKIP] 5/7036 - ASIN B000056HNO already processed.
[SKIP] 6/7036 - ASIN B000056J23 already processed.
[SKIP] 7/7036 - ASIN B000056J90 already processed.
[SKIP] 8/7036 - ASIN B000056JEG already processed.
[SKIP] 9/7036 - ASIN B000056JLK already processed.
[SKIP] 10/7036 - ASIN B000056OV0 already processed.
[SKIP] 11/7036 - ASIN B000056W6F already processed.
[SKIP] 12/7036 - ASIN B000058DN9 already processed.
[SKIP] 13/7036 - ASIN B000059XP6 already processed.
[SKIP] 14/7036 - ASIN B00005BQ4N already processed.
[SKIP] 15/7036 - ASIN B00005BTNA already processed.
[SKIP] 16/7036 - ASIN B00005BYUJ already processed.
[SKIP] 17/7036 - ASIN B000

In [ ]:

import os
import json
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_deepseek import ChatDeepSeek  # 👈 DeepSeek integration


# ---------------------------
# SETUP LLM + PARSER (DEEPSEEK)
# ---------------------------
DEEPSEEK_API_KEY = os.environ.get("DEEPSEEK_API_KEY")  # or hardcode for quick test

llm = ChatDeepSeek(
    model="deepseek-chat",                 # or "deepseek-reasoner" if you want CoT
    api_key=DEEPSEEK_API_KEY,
    api_base="https://api.deepseek.com",
    temperature=0,
)

parser = StrOutputParser()




# ---------------------------
# PHASE 1: ANALYSIS
# ---------------------------
analysis_prompt = PromptTemplate(
    input_variables=["title", "img_title_desc", "original_desc"],
    template="""
You are an information extraction assistant.

Product title:
{title}

Descriptions:
1. Image-title-based description: {img_title_desc}
2. Original Amazon description: {original_desc}

Your tasks:
1. Identify all common correct details appearing in both descriptions or implied by the title.
2. Identify useful unique details from each description.
3. Identify contradictions between them.

Return JSON EXACTLY:

{{
  "common_details": [...],
  "unique_details": {{
    "img_title_desc": [...],
    "original_desc": [...]
  }},
  "contradictions": [...]
}}
"""
)
analysis_chain = analysis_prompt | llm | parser


rewrite_prompt = PromptTemplate(
    input_variables=["title", "analysis", "feedback"],
    template="""
You are a product description rewriting assistant.

Product title:
{title}

Here is a JSON analysis of this product:
{analysis}

Here is feedback from fact-checking (if any):
{feedback}

RULES ABOUT SOURCE PRIORITY:
- The product title defines the core product identity.
- The original description is the most authoritative source of factual information.
- The image_title_desc may be used for tone, phrasing, and soft implications, but NOT to override or contradict the original_desc.
- The image_based_desc should generally NOT be used for factual claims unless it clearly matches the product type.

Your job:
Write or revise ONE unified product description (1–3 sentences) that:
- is clearly about the product in the title,
- preserves facts from the original_desc first,
- may use wording or sentiment from img_title_desc to make the description more natural,
- removes or softens unsupported or conflicting claims,
- does NOT invent new facts or assume details.

CRITICAL RULES FOR CONFLICTING INFORMATION:
- If the analysis lists a contradiction, do not choose one side. Use neutral grouping phrasing.
  Example:
    WRONG: “includes a pacifier, gum massager, and a teether”
    CORRECT: “includes multiple oral care tools designed for early dental hygiene”

- If the analysis lists a contradiction in material:
    WRONG: “100% cotton”
    WRONG: “cotton blend”
    CORRECT: “cotton fabric”
    CORRECT: “soft cotton material”

Fallback rules:
- If original_desc is missing or extremely short, you may rely more on img_title_desc but keep claims conservative and neutral.
- If there is uncertainty, write a safer, more general phrasing rather than a strong factual claim.

Requirements:
- Start the description with the exact product name from the title.
- Make the text natural and suitable for an online product listing.
- Apply feedback faithfully: if a detail was flagged as unsupported, do NOT reintroduce it.

Return ONLY the final product description as plain text.
"""
)

rewrite_chain = rewrite_prompt | llm | parser



reflection_prompt = PromptTemplate(
    input_variables=["title", "draft", "img_title_desc", "original_desc"],
    template="""
You are a fact-checking and consistency validation agent.

Product title:
{title}

Draft fused description:
{draft}

Allowed sources for verification (in descending reliability):
Tier 1 (highest trust): Product title
Tier 2: Original description
Tier 3: Image_title_desc (linguistic tone & soft implications, NOT technical facts)
Tier 4 (lowest trust): image_based_desc — use only for confirming object category, NOT specific details.

Your goal is NOT to be overly strict, but to verify that the draft is:
- factually safe,
- respects the source priority order,
- conflict-neutral when sources disagree,
- and does not assert unsupported strong claims.

RULES:

1) A statement in the draft is SUPPORTED if it is:
- explicitly stated in the original_desc or title, OR
- clearly implied by them, OR
- consistent with img_title_desc phrasing (but NOT contradictory to original_desc), OR
- a neutral/general phrasing that does not choose sides in a contradiction.

2) If the sources contradict each other about some detail (such as included components or material), then:
- Strong specific claims are UNSUPPORTED.
- Neutral grouping phrasing like “includes several items for…” is ACCEPTABLE.

3) The image_based_desc is NOT sufficient to reject or validate facts UNLESS it contradicts product identity in title.
Example:
title: toothbrush
image shows: smartphone
→ image should be ignored

4) Only flag a FAIL if:
- the draft contains a strong factual claim
- AND that exact claim is NOT supported by the original_desc or title,
- AND the claim is not a neutral/general phrasing,
- AND the claim is not simply stylistic paraphrasing.

5) You MUST NOT fail the draft:
- for neutral phrasing,
- or for safe paraphrasing,
- or for stylistic differences,
- or for soft/implicit wording of common-sense facts,
- or for minor differences in tone or emphasis.

OUTPUT FORMAT:

If all facts in the draft are supported or neutrally phrased → return EXACTLY:

OK

Otherwise return:

FAIL: [list of specific exact problematic phrases]

where each phrase appears literally in the draft text.
"""
)

reflection_chain = reflection_prompt | llm | parser








# ---------------------------
# TEST ITEM
# ---------------------------
item = {
  "asin": "XTEST02",
  "title": "Summer Infant 3 Piece Oral Care Set",
  "image_title_based_desc": "Includes soft-bristled toothbrush, teething ring, and pacifier, designed for oral development from infancy.",
  "original_description": "Kit includes a 3 stage oral care system: gum massager, finger toothbrush, and toddler toothbrush. Helps parents maintain oral hygiene.",
  "image_based_desc": "The product is a small boxed kit containing multiple small plastic baby care items.",
  "llm_based_desc": "The Summer Infant 3 Piece Oral Care Set features teethers and pacifiers for soothing, along with a toothbrush designed for baby gums."
}



import re
import json

def extract_json(text: str) -> str:
    """
    Remove ```json ... ``` or ``` ... ``` fences and return the inner JSON string.
    """
    text = text.strip()
    # If it starts with ``` something, strip the code fence
    if text.startswith("```"):
        # Remove leading ```... and trailing ```
        text = re.sub(r"^```[a-zA-Z]*\n?", "", text)   # remove ```json\n or ```\n
        text = re.sub(r"\n?```$", "", text.strip())   # remove closing ```
    return text.strip()

# ---------------------------
# SIMPLE TEST PIPELINE
# ---------------------------
def run_deepseek_test(item, max_loops=2):
    title = item["title"]
    img_title_desc = item["image_title_based_desc"]
    original_desc = item["original_description"]

    print("\n==============================")
    print(f"RUNNING ITEM: {title}")
    print("==============================")

    # ---- Phase 1: ANALYSIS ----
    analysis_raw = analysis_chain.invoke({
    "title": title,
    "img_title_desc": img_title_desc,
    "original_desc": original_desc
    })
    print("\n--- PHASE 1 ANALYSIS (RAW) ---")
    print(analysis_raw)

    try:
        cleaned = extract_json(analysis_raw)
        analysis_json = json.loads(cleaned)
    except json.JSONDecodeError:
        print("\n[WARN] DeepSeek returned invalid JSON even after cleaning. Using raw string for next phase.")
        analysis_json = analysis_raw


    feedback = "No feedback yet. This is the first draft."
    last_draft = None

    # ---- Looped REWRITE ↔ REFLECTION ----
    for i in range(max_loops):
        print(f"\n=== LOOP TURN {i+1} ===")

        # Phase 2: REWRITE
        draft = rewrite_chain.invoke({
            "title": title,
            "analysis": analysis_json if isinstance(analysis_json, str) else json.dumps(analysis_json),
            "feedback": feedback,
        })
        print("\n--- PHASE 2 DRAFT ---")
        print(draft)
        last_draft = draft

        # Phase 3: REFLECTION
        reflection = reflection_chain.invoke({
            "title": title,
            "draft": draft,
            "img_title_desc": img_title_desc,
            "original_desc": original_desc
        })
        print("\n--- PHASE 3 REFLECTION ---")
        print(reflection)

        if reflection.strip().startswith("OK"):
            print("\n✅ Reflection says OK. Stopping loop.")
            break
        else:
            feedback = reflection  # feed FAIL back into next rewrite

    return last_draft, reflection


if __name__ == "__main__":
    final_draft, final_reflection = run_deepseek_test(item)
    print("\n==============================")
    print("FINAL DRAFT:")
    print(final_draft)
    print("\nFINAL REFLECTION:")
    print(final_reflection)

     


RUNNING ITEM: Summer Infant 3 Piece Oral Care Set

--- PHASE 1 ANALYSIS (RAW) ---
```json
{
  "common_details": ["3-piece set", "oral care", "infant/toddler use", "multiple oral care items"],
  "unique_details": {
    "img_title_desc": ["soft-bristled toothbrush", "teething ring", "pacifier", "designed for oral development from infancy"],
    "original_desc": ["3 stage oral care system", "gum massager", "finger toothbrush", "toddler toothbrush", "helps parents maintain oral hygiene"]
  },
  "contradictions": ["The specific items included are contradictory: image-title description lists toothbrush, teething ring, and pacifier, while original description lists gum massager, finger toothbrush, and toddler toothbrush"]
}
```

=== LOOP TURN 1 ===

--- PHASE 2 DRAFT ---
Summer Infant 3 Piece Oral Care Set is a 3-stage system designed to help parents maintain their infant's oral hygiene from infancy through toddlerhood. The set includes multiple oral care tools such as a gum massager, a fing